# L3 M8.4: Temporal Financial Information Handling

## Learning Arc

**Core Problem**: When someone says "Q3 2024," they're referring to different calendar months depending on company fiscal year ends:
- **Apple** (FY ends Sept 30): Q3 FY2024 = April 1 - June 30, 2024
- **Microsoft** (FY ends June 30): Q3 FY2024 = January 1 - March 31, 2024
- **Walmart** (FY ends Jan 31): Q3 FY2024 = August 1 - October 31, 2023

### What You'll Build

Production-ready temporal financial retrieval system that:
1. Maps 20+ companies to their fiscal year ends
2. Converts fiscal quarters to calendar dates (100% accuracy)
3. Implements point-in-time queries ("as of March 15, 2023")
4. Validates temporal consistency across documents
5. Handles forward-looking statement decay

### By the End

You will:
- ✅ Understand why fiscal periods are company-specific
- ✅ Implement fiscal → calendar date conversion
- ✅ Build temporal filters for vector database queries
- ✅ Validate temporal consistency (prevent mixing FY2023 + FY2024 data)
- ✅ Establish compliance framework (SOX, GAAP, Reg FD)

### Key Takeaway

**Explicit metadata filtering beats semantic search for temporal accuracy in financial systems.**

## Environment Check

In [ ]:
import os
import sys

# Add parent directory to path for imports
sys.path.insert(0, '..')

# Check service configuration
PINECONE_ENABLED = os.getenv("PINECONE_ENABLED", "false").lower() == "true"
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY", "")

ANTHROPIC_ENABLED = os.getenv("ANTHROPIC_ENABLED", "false").lower() == "true"
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "")

print("="*60)
print("L3 M8.4: Temporal Financial Information Handling")
print("="*60)
print(f"\nPinecone:   {'✅ Enabled' if PINECONE_ENABLED else '⚠️  Disabled'}")
if PINECONE_ENABLED:
    print(f"  API Key:  {'✅ Set' if PINECONE_API_KEY else '❌ Not Set'}")
    if not PINECONE_API_KEY:
        print("  ⚠️  Network calls will fail. Set PINECONE_API_KEY environment variable.")
else:
    print("  ℹ️  Notebook will skip vector database queries.")
    print("     To enable: Set PINECONE_ENABLED=true in environment")

print(f"\nAnthropic:  {'✅ Enabled' if ANTHROPIC_ENABLED else '⚠️  Disabled (Optional)'}")
if ANTHROPIC_ENABLED:
    print(f"  API Key:  {'✅ Set' if ANTHROPIC_API_KEY else '❌ Not Set'}")

print("\n" + "="*60)
print("✅ Environment check complete")
print("="*60 + "\n")

## 1. Introduction & Conceptual Foundation

### The Fiscal Period Problem

Financial analysts think in **fiscal quarters**, but:
- Companies have different fiscal year ends
- "Q3 FY2024" means different calendar dates for different companies
- Vector databases filter by **calendar dates**, not fiscal periods

**Solution**: Build a fiscal calendar mapping system that converts fiscal periods to calendar dates.

In [ ]:
# Import core modules
from src.l3_m8_financial_domain_knowledge_injection import (
    FiscalCalendarManager,
    TemporalRetriever,
    TemporalValidator,
    fiscal_quarter_to_dates
)

print("✅ Imported temporal financial modules")

### Key Concepts

**1. Fiscal Year vs Calendar Year**
- Calendar year: January 1 - December 31
- Fiscal year: Company-specific (e.g., Apple: Oct 1 - Sept 30)

**2. Point-in-Time Queries**
- Reconstruct historical information state
- "What was Apple's revenue as of March 15, 2023?"
- Filter documents filed **before** that date

**3. Temporal Consistency**
- Never mix data from different fiscal periods
- FY2023 revenue + FY2024 expenses = **invalid** financial ratios

## 2. Fiscal Calendar Manager

### Initialize Fiscal Calendar Database

Loads fiscal year end dates for 20+ companies from `data/fiscal_year_ends.json`

In [ ]:
# Initialize fiscal calendar manager
fiscal_manager = FiscalCalendarManager()

print(f"Loaded fiscal year data for {len(fiscal_manager.fiscal_year_ends)} companies")
print("\nSample companies:")
for ticker in list(fiscal_manager.fiscal_year_ends.keys())[:5]:
    data = fiscal_manager.fiscal_year_ends[ticker]
    print(f"  {ticker:6} - {data['company_name']:30} (FY ends {data['fiscal_year_end']})")

### Lookup Fiscal Year End for Companies

In [ ]:
# Get fiscal year end for specific companies
companies = ['AAPL', 'MSFT', 'WMT', 'AMZN', 'GOOGL']

print("Fiscal Year Ends:")
print("-" * 70)
for ticker in companies:
    data = fiscal_manager.get_fiscal_year_end(ticker)
    if data:
        print(f"{ticker:6} | {data['company_name']:25} | FY ends: {data['fiscal_year_end']}")
    else:
        print(f"{ticker:6} | Not found in fiscal database")

# Expected:
# AAPL   | Apple Inc.                | FY ends: 09-30
# MSFT   | Microsoft Corporation     | FY ends: 06-30
# WMT    | Walmart Inc.              | FY ends: 01-31

## 3. Fiscal Quarter to Calendar Date Conversion

### Core Temporal Logic

**Algorithm**: Works backward from fiscal year end
- Q4 ends on FY end date
- Q3 ends 3 months before Q4
- Q2 ends 6 months before Q4
- Q1 ends 9 months before Q4

In [ ]:
# Convert Apple Q3 FY2024 to calendar dates
ticker = 'AAPL'
fiscal_year = 2024
quarter = 'Q3'

start_date, end_date = fiscal_manager.fiscal_quarter_to_dates(ticker, fiscal_year, quarter)

print(f"{ticker} {quarter} FY{fiscal_year}:")
print(f"  Calendar dates: {start_date} to {end_date}")
print(f"  Duration: 3 months (April - June 2024)")

# Expected: 2024-04-01 to 2024-06-30

### Compare Different Companies' Q3 FY2024

**Demonstrates**: Same fiscal quarter = different calendar periods

In [ ]:
# Compare Q3 FY2024 across different companies
companies = ['AAPL', 'MSFT', 'WMT']
quarter = 'Q3'
fiscal_year = 2024

print(f"Q3 FY{fiscal_year} Calendar Date Comparison:")
print("="*80)
for ticker in companies:
    dates = fiscal_manager.fiscal_quarter_to_dates(ticker, fiscal_year, quarter)
    if dates:
        start, end = dates
        company_data = fiscal_manager.get_fiscal_year_end(ticker)
        print(f"{ticker:6} (FY ends {company_data['fiscal_year_end']}): {start} to {end}")

print("\n⚠️  Notice: All are 'Q3 FY2024' but different calendar periods!")
print("   This is why explicit fiscal mapping is critical for accuracy.")

# Expected:
# AAPL   (FY ends 09-30): 2024-04-01 to 2024-06-30  (April-June)
# MSFT   (FY ends 06-30): 2024-01-01 to 2024-03-31  (Jan-March)
# WMT    (FY ends 01-31): 2023-08-01 to 2023-10-31  (Aug-Oct 2023!)

### Test All Quarters for Apple FY2024

In [ ]:
# Show all 4 quarters for Apple FY2024
ticker = 'AAPL'
fiscal_year = 2024

print(f"{ticker} Fiscal Year {fiscal_year} (FY ends Sept 30):")
print("="*70)
for q in ['Q1', 'Q2', 'Q3', 'Q4']:
    start, end = fiscal_manager.fiscal_quarter_to_dates(ticker, fiscal_year, q)
    print(f"  {q} FY{fiscal_year}: {start} to {end}")

print("\nℹ️  Notice: Apple's FY2024 spans Oct 2023 - Sept 2024 (two calendar years)")

# Expected:
# Q1 FY2024: 2023-10-01 to 2023-12-31  (Oct-Dec 2023)
# Q2 FY2024: 2024-01-01 to 2024-03-31  (Jan-Mar 2024)
# Q3 FY2024: 2024-04-01 to 2024-06-30  (Apr-Jun 2024)
# Q4 FY2024: 2024-07-01 to 2024-09-30  (Jul-Sep 2024)

## 4. Temporal Retriever

### Initialize Temporal Retriever

Integrates fiscal calendar with vector database queries (simulated without Pinecone client)

In [ ]:
# Initialize temporal retriever (without vector client)
temporal_retriever = TemporalRetriever(fiscal_manager, vector_client=None)

print("✅ Initialized TemporalRetriever")
print("   Vector client: None (simulation mode)")
print("   To enable actual queries: Set PINECONE_ENABLED=true and configure client")

### Query Fiscal Period

**System Flow:**
1. Convert fiscal period to calendar dates
2. Vector search with metadata filter: `ticker=AAPL AND filing_date BETWEEN start AND end`
3. Validate temporal consistency
4. Return temporally-verified results

In [ ]:
# Query Apple Q3 FY2024 for revenue information
result = temporal_retriever.query_fiscal_period(
    ticker='AAPL',
    fiscal_year=2024,
    quarter='Q3',
    query_text='revenue growth',
    top_k=5
)

print("Query Result:")
print("-" * 70)
print(f"Status:         {result['status']}")
print(f"Ticker:         {result['ticker']}")
print(f"Fiscal Period:  {result['fiscal_period']}")
print(f"Calendar Period: {result['calendar_period']}")
print(f"Query:          {result['query']}")

if 'metadata' in result:
    print(f"\nMetadata Filter Applied:")
    print(f"  {result['metadata'].get('filter_applied', 'N/A')}")

# Expected: Converts Q3 FY2024 to calendar dates and shows filter

## 5. Point-in-Time Queries

### Reconstruct Historical Information State

**Use Case**: "What was Apple's revenue as of March 15, 2023?"

Filters documents filed **before** the specified date

In [ ]:
# Point-in-time query: Apple as of March 15, 2023
result = temporal_retriever.point_in_time_query(
    ticker='AAPL',
    as_of_date='2023-03-15',
    query_text='What was Apple\'s revenue?',
    top_k=5
)

print("Point-in-Time Query Result:")
print("-" * 70)
print(f"Status:       {result['status']}")
print(f"Ticker:       {result['ticker']}")
print(f"As of Date:   {result['as_of_date']}")
print(f"Query:        {result['query']}")

if 'metadata' in result:
    print(f"\nMetadata Filter Applied:")
    print(f"  {result['metadata'].get('filter_applied', 'N/A')}")

print("\nℹ️  Only documents filed before 2023-03-15 would be retrieved")

# Expected: Shows filter filing_date <= 2023-03-15

### Multiple Point-in-Time Queries

In [ ]:
# Compare information state at different points in time
dates = ['2022-12-31', '2023-06-30', '2024-01-15']

print("Apple Revenue Information State Over Time:")
print("="*70)
for as_of_date in dates:
    result = temporal_retriever.point_in_time_query(
        ticker='AAPL',
        as_of_date=as_of_date,
        query_text='revenue',
        top_k=3
    )
    print(f"\nAs of {as_of_date}:")
    print(f"  Status: {result['status']}")
    if 'metadata' in result:
        print(f"  Filter: {result['metadata'].get('filter_applied', 'N/A')}")

print("\nℹ️  Each query reconstructs the information state at that specific date")

## 6. Temporal Consistency Validation

### Prevent Invalid Cross-Period Comparisons

**Problem**: Mixing FY2023 revenue + FY2024 expenses = **invalid** financial ratios

**Solution**: Validate all documents are from same fiscal period

In [ ]:
# Initialize temporal validator
temporal_validator = TemporalValidator(fiscal_manager)

print("✅ Initialized TemporalValidator")

### Test Valid Documents (Same Fiscal Period)

In [ ]:
# Valid: All documents from Apple Q3 FY2024
valid_documents = [
    {"ticker": "AAPL", "filing_date": "2024-04-15", "fiscal_period": "Q3 FY2024"},
    {"ticker": "AAPL", "filing_date": "2024-05-20", "fiscal_period": "Q3 FY2024"},
    {"ticker": "AAPL", "filing_date": "2024-06-10", "fiscal_period": "Q3 FY2024"}
]

result = temporal_validator.validate_temporal_consistency(valid_documents, strict=True)

print("Validation Result (Valid Documents):")
print("-" * 70)
print(f"Status:         {result['status']}")
print(f"Issues Found:   {len(result['issues'])}")
print(f"Document Count: {result['summary']['document_count']}")
print(f"Tickers:        {result['summary']['tickers']}")
print(f"Fiscal Periods: {result['summary']['fiscal_periods']}")

# Expected: status = 'valid', 0 issues

### Test Invalid Documents (Mixed Fiscal Periods)

In [ ]:
# Invalid: Mixing Q3 and Q4 FY2024
invalid_documents = [
    {"ticker": "AAPL", "filing_date": "2024-04-15", "fiscal_period": "Q3 FY2024"},
    {"ticker": "AAPL", "filing_date": "2024-07-15", "fiscal_period": "Q4 FY2024"}  # Different quarter!
]

result = temporal_validator.validate_temporal_consistency(invalid_documents, strict=True)

print("Validation Result (Invalid - Mixed Periods):")
print("-" * 70)
print(f"Status:         {result['status']}")
print(f"Issues Found:   {len(result['issues'])}")
print("\nIssues:")
for issue in result['issues']:
    print(f"  [{issue['severity'].upper()}] {issue['type']}: {issue['message']}")

# Expected: status = 'invalid', mixed_fiscal_periods error

### Test Warning: Large Date Range

In [ ]:
# Warning: Documents span > 6 months (potential staleness)
stale_documents = [
    {"ticker": "AAPL", "filing_date": "2024-01-15", "fiscal_period": "Q1 FY2024"},
    {"ticker": "AAPL", "filing_date": "2024-09-15", "fiscal_period": "Q4 FY2024"}
]

result = temporal_validator.validate_temporal_consistency(stale_documents, strict=False)

print("Validation Result (Large Date Range):")
print("-" * 70)
print(f"Status:         {result['status']}")
print(f"Issues Found:   {len(result['issues'])}")
print("\nIssues/Warnings:")
for issue in result['issues']:
    print(f"  [{issue['severity'].upper()}] {issue['type']}: {issue['message']}")

# Expected: large_date_range warning

## 7. Forward-Looking Statement Assessment

### Confidence Decay for Guidance

**Problem**: Guidance from Q1 2024 may be invalid by Q4 2024

**Solution**: Add "confidence decay" based on statement age

In [ ]:
# Recent forward-looking statement (high confidence)
recent_statement = {
    "filing_date": "2024-10-01",
    "is_forward_looking": True
}

result = temporal_validator.check_forward_looking_statements(
    recent_statement,
    current_date="2024-11-01"
)

print("Forward-Looking Statement Assessment (Recent):")
print("-" * 70)
print(f"Status:     {result['status']}")
print(f"Confidence: {result['confidence']}")
print(f"Age (days): {result['age_days']}")
print(f"Message:    {result['message']}")

# Expected: confidence = 'high' (< 3 months)

In [ ]:
# Stale forward-looking statement (low confidence)
stale_statement = {
    "filing_date": "2023-01-01",
    "is_forward_looking": True
}

result = temporal_validator.check_forward_looking_statements(
    stale_statement,
    current_date="2024-01-01"
)

print("Forward-Looking Statement Assessment (Stale):")
print("-" * 70)
print(f"Status:     {result['status']}")
print(f"Confidence: {result['confidence']}")
print(f"Age (days): {result['age_days']}")
print(f"Message:    {result['message']}")

# Expected: confidence = 'low' (> 6 months)

## 8. Integration with Entity Linking (M8.3)

### End-to-End Flow

```
User Query: "What was Microsoft's Q2 revenue?"
       ↓
[M8.3 Entity Linking]: "Microsoft" → ticker: MSFT
       ↓
[M8.4 Fiscal Mapping]: MSFT Q2 FY2024 → Oct 1 - Dec 31, 2023
       ↓
[Vector DB Query]: ticker=MSFT AND filing_date BETWEEN 2023-10-01 AND 2023-12-31
```

In [ ]:
# Simulate entity linking → fiscal mapping flow
user_query = "What was Microsoft's Q2 FY2024 revenue?"

# Step 1: Entity Linking (M8.3) - simulated
company_name = "Microsoft"
ticker = "MSFT"  # From entity linking
fiscal_year = 2024
quarter = "Q2"

print(f"User Query: {user_query}")
print("-" * 70)
print(f"\n[Step 1] Entity Linking (M8.3):")
print(f"  '{company_name}' → ticker: {ticker}")

# Step 2: Fiscal Period Mapping (M8.4)
start_date, end_date = fiscal_manager.fiscal_quarter_to_dates(ticker, fiscal_year, quarter)
print(f"\n[Step 2] Fiscal Period Mapping (M8.4):")
print(f"  {ticker} {quarter} FY{fiscal_year} → {start_date} to {end_date}")

# Step 3: Vector DB Query (with temporal filter)
print(f"\n[Step 3] Vector DB Query:")
print(f"  Filter: ticker={ticker} AND filing_date BETWEEN {start_date} AND {end_date}")

print(f"\n✅ Complete temporal-aware retrieval flow")

# Expected: MSFT Q2 FY2024 = Oct 1 - Dec 31, 2023

## 9. Common Failure Modes

### Failure Scenarios from Production

Based on real-world financial system deployments

In [ ]:
# Failure #1: Fiscal Year Database Out of Date
print("Failure #1: Fiscal Year Database Out of Date")
print("="*70)
print("Problem: Companies occasionally change fiscal year ends")
print("Detection: Query returns unexpected date ranges")
print("Mitigation:")
print("  - Update data/fiscal_year_ends.json annually")
print("  - Version control fiscal year database")
print("  - Add 'last_updated' field to track data freshness")
print("  - Subscribe to SEC Edgar RSS feeds for FY changes\n")

# Failure #2: Cross-Company Fiscal Period Mismatch
print("Failure #2: Cross-Company Fiscal Period Mismatch")
print("="*70)
print("Problem: Comparing Apple Q3 to Microsoft Q3 from different calendar periods")
print("Detection: TemporalValidator returns 'mixed_fiscal_periods' error")
print("Mitigation:")
print("  - Use validate_temporal_consistency() with strict=True")
print("  - Normalize to calendar quarters for cross-company analysis")
print("  - Add warnings when comparing different fiscal periods\n")

# Failure #3: Forward-Looking Statement Becomes Outdated
print("Failure #3: Forward-Looking Statement Becomes Outdated")
print("="*70)
print("Problem: Guidance from Q1 2024 may be invalid by Q4 2024")
print("Detection: Forward-looking statements used after 6+ months")
print("Mitigation:")
print("  - Add 'confidence decay' metadata")
print("  - Flag statements > 6 months old as 'potentially outdated'")
print("  - Use check_forward_looking_statements() validation")

## 10. Production Deployment Considerations

### Compliance Requirements (SOX, GAAP, Reg FD)

In [ ]:
print("Production Deployment Checklist:")
print("="*70)
checklist = [
    "Fiscal year database covers 95%+ of portfolio companies",
    "All documents tagged with filing_date and report_date",
    "Temporal validation tests pass 100% accuracy",
    "Audit trail logs all fiscal period conversions",
    "Disclaimers included in all API responses"
]

for i, item in enumerate(checklist, 1):
    print(f"  [{' ' if i > 0 else '✓'}] {item}")

print("\nRequired Disclaimers:")
print("-" * 70)
disclaimer = (
    "Data retrieved from {Company} {Fiscal Period} ({Calendar Dates}). "
    "Historical data; not real-time. See original filings for authoritative information."
)
print(f"  '{disclaimer}'")

print("\nRegulatory Compliance:")
print("-" * 70)
print("  - SOX Section 302: CEO/CFO certification based on 'as of' dates")
print("  - SOX Section 404: Audit trails for financial data retrieval")
print("  - GAAP: Consistent fiscal periods for comparisons")
print("  - Regulation FD: Uniform disclosure dates tracked by filing_date")

### Performance & Cost Optimization

In [ ]:
print("Cost Optimization Strategies:")
print("="*70)
strategies = [
    "Cache fiscal year ends in Redis (lookup latency: 1ms vs 50ms)",
    "Batch fiscal period conversions (convert Q1-Q4 upfront)",
    "Use metadata-only filtering (don't retrieve full documents)",
    "Shared fiscal calendar across teams"
]

for strategy in strategies:
    print(f"  ✓ {strategy}")

print("\nROI Calculation:")
print("-" * 70)
print("  Cost of one erroneous investment decision: $1M+")
print("  Temporal accuracy preventing 1 error/year: ROI = 100-200x")
print("  \n  → Temporal accuracy is a critical risk mitigation investment")

## 11. Summary & Next Steps

### What You Accomplished

✅ Built fiscal calendar system mapping companies to fiscal year ends

✅ Implemented fiscal → calendar date conversion (core temporal logic)

✅ Integrated with entity linking (M8.3) for company resolution

✅ Validated temporal consistency across retrieval results

✅ Established production compliance framework (SOX, GAAP, Reg FD)

### Key Takeaways

- **Fiscal periods are company-specific**; never assume calendar year
- **Explicit metadata filtering** beats semantic search for temporal accuracy
- **Point-in-time queries** require document `filing_date` tracking
- **Forward-looking statements** have built-in expiration dates
- **Financial systems demand 100% accuracy**; test every date conversion

### What's Next: Finance AI M9.1

**Multi-source financial data fusion**: Combine SEC filings + earnings calls + market data

**Before Next Video**:
- Extend your fiscal calendar to 50+ companies
- Add historical fiscal year changes (companies occasionally change FY ends)
- Integrate with real filing data from SEC EDGAR

## Resources

### Documentation
- **SEC EDGAR**: https://www.sec.gov/cgi-bin/browse-edgar (authoritative fiscal year sources)
- **XBRL Instance Documents**: Machine-readable fiscal period metadata
- **Company Investor Relations**: Fiscal calendar documentation

### Related Modules
- **M8.1**: Financial document types and regulatory context
- **M8.2**: Real-time financial data caching
- **M8.3**: Financial entity linking (company name → ticker)
- **M9.1**: Multi-source financial data fusion (next module)

### External Tools
- **Pinecone**: https://www.pinecone.io/docs/ (vector database)
- **Anthropic Claude**: https://docs.anthropic.com/ (LLM generation)
- **Python dateutil**: https://dateutil.readthedocs.io/ (fiscal period calculations)